In [4]:
from bs4 import BeautifulSoup
import requests

import pandas as pd
import os


In [5]:
def scraped_file_loc(file_name):
    return os.path.join(SCRAPED_DATA_FILE_LOCATION, file_name) + '.csv'

In [ ]:
from lxml import etree

## Parse grants.gov xml to csv delimited by ~

http://www.grants.gov/help/html/help/index.htm#t=XML_Extract%2FXML_Extract.htm

http://www.grants.gov/web/grants/xml-extract.html

http://www.grants.gov/web/grants/xml-extract.html?p_p_id=xmlextract_WAR_grantsxmlextractportlet_INSTANCE_5NxW0PeTnSUa&p_p_lifecycle=2&p_p_state=normal&p_p_mode=view&p_p_cacheability=cacheLevelPage&p_p_col_id=column-1&p_p_col_pos=1&p_p_col_count=2&download=GrantsDBExtract20170210v2.zip

http://www.grants.gov/custom/viewOppDetails.jsp?oppId=196633

In [327]:
f = open('grants_gov_xml/GrantsDBExtract20170210v2.xml', 'r')
#     bs = BeautifulSoup(f)

# context = etree.iterparse(f)

In [328]:
root = etree.parse(f)

In [339]:
COMMON_PREFIX = '{http://apply.grants.gov/system/OpportunityDetail-V1.0}'

In [341]:
grant_trees = root.findall('{http://apply.grants.gov/system/OpportunityDetail-V1.0}OpportunitySynopsisDetail_1_0')

In [388]:
headers

['OpportunityID',
 'OpportunityTitle',
 'OpportunityNumber',
 'OpportunityCategory',
 'FundingInstrumentType',
 'CategoryOfFundingActivity',
 'CategoryExplanation',
 'CFDANumbers',
 'EligibleApplicants',
 'AdditionalInformationOnEligibility',
 'AgencyCode',
 'AgencyName',
 'PostDate',
 'CloseDate',
 'LastUpdatedDate',
 'AwardCeiling',
 'AwardFloor',
 'EstimatedTotalProgramFunding',
 'ExpectedNumberOfAwards',
 'Description',
 'Version',
 'CostSharingOrMatchingRequirement',
 'ArchiveDate',
 'GrantorContactEmail',
 'GrantorContactEmailDescription',
 'GrantorContactText',
 'AdditionalInformationURL',
 'AdditionalInformationText',
 'OpportunityCategoryExplanation',
 'CloseDateExplanation']

In [381]:
rows = []
headers = [ele.tag[len(COMMON_PREFIX):] for ele in grant_trees[0]]

multiple_cols = set()

for grant in grant_trees:
    d = {}
    for ele in grant:
        tag = ele.tag[len(COMMON_PREFIX):]
        if tag not in headers:
            headers.append(tag)
        
        if tag not in d:
            d[tag] = [ele.text]
        else:
            d[tag].append(ele.text)
    
    row = []
    for header in headers:
        if header not in d:
            row.append(None)
        else:
            if len(d[header]) > 1:
                multiple_cols.add(header)
            row.append(','.join(d[header]))
    rows.append(row)

In [382]:
print multiple_cols

set(['CategoryOfFundingActivity', 'EligibleApplicants', 'CFDANumbers', 'FundingInstrumentType'])


In [383]:
len(rows)

41536

In [384]:
df = pd.DataFrame(rows, columns=headers)

In [386]:
df.to_csv(scraped_file_loc('grants_gov'), sep='~', index=False)

In [387]:
pd.read_csv(scraped_file_loc('grants_gov'), sep='~')

,OpportunityID,OpportunityTitle,OpportunityNumber,OpportunityCategory,FundingInstrumentType,CategoryOfFundingActivity,CategoryExplanation,CFDANumbers,EligibleApplicants,AdditionalInformationOnEligibility,...,Version,CostSharingOrMatchingRequirement,ArchiveDate,GrantorContactEmail,GrantorContactEmailDescription,GrantorContactText,AdditionalInformationURL,AdditionalInformationText,OpportunityCategoryExplanation,CloseDateExplanation
0,262148,Establishment of the Edmund S. Muskie Graduate...,SCAPPD-14-AW-161-SCA-08152014,D,CA,O,Public Diplomacy,19.040,25,Eligibility for U.S. institutions is limited t...,...,Synopsis 2,No,10042014.0,DelageEM@state.gov,Inquiries,"Ellen Delage, Bureau of South and Central Asia...",NaN,NaN,NaN,NaN
1,262149,Eradication of Yellow Crazy Ants on Johnston A...,F14AS00402,D,CA,"AG,ENV,NR",NaN,15.608,99,The recipient has already been selected for th...,...,Synopsis 1,No,8232014.0,Connie_Sauer@fws.gov,Connie_Sauer@fws.gov,"Connie Sauer, 503-490-0827&lt;br/&gt;Connie_Sa...",http://www.grants.gov/,http://www.grants.gov/,NaN,NaN
2,131073,"Cooperative Ecosystem Studies Unit, Piedmont S...",G12AS20003,D,CA,ST,NaN,15.808,25,This financial assistance opportunity is being...,...,Synopsis 2,No,12172011.0,fgraves@usgs.gov,fgraves@usgs.gov,"Faith Graves, 703-648-7356&lt;br/&gt;fgraves@u...",http://www.grants.gov/,http://www.grants.gov/,NaN,NaN
3,196613,OVW FY 2012 Training and Technical Assistance ...,OVW-2012-3380,D,CA,LJL,NaN,16.526,"06,12",NaN,...,Synopsis 1,No,11142012.0,Christina.Murray@usdoj.gov,Christina.Murray@usdoj.gov,Christina Murray &lt;br/&gt;Phone 202-307-0344,http://www.ovw.usdoj.gov/docs/grantee-reportin...,OVW FY 2012 Training and Technical Assistance ...,CategoryExplanation,NaN
4,65553,Affordable Care Act (ACA): Childhood Obesity R...,RFA-DP-11-007,D,CA,HL,NaN,93.535,99,NaN,...,Synopsis 10,No,5182011.0,pgotim@cdc.gov,General Email,Centers for Disease Control and Prevention (CD...,Please click on Full Announcement at the top o...,Please click on Full Announcement at the top o...,NaN,On-time submission requires that electronic ap...
5,65554,Ancillary Studies in Clinical Trials (R01),RFA-HL-12-012,D,G,HL,NaN,"93.233,93.837,93.838,93.839","00,01,02,04,05,06,07,08,11,12,13,20,22,23,25",Other Eligible Applicants include the followin...,...,Synopsis 1,No,2272012.0,FBOWebmaster@OD.NIH.GOV,If you have any problems linking to this fundi...,NIH OER Webmaster&lt;br/&gt;FBOWebmaster@OD.NI...,http://grants.nih.gov/grants/guide/rfa-files/R...,http://grants.nih.gov/grants/guide/rfa-files/R...,NaN,NaN
6,131094,Plant Feedstock Genomics for Bioenergy: A Joi...,DE-FOA-0000598,D,G,ST,NaN,81.049,99,DOE Eligibility Criteria: Applicants from U.S....,...,Synopsis 1,No,5242012.0,kim.laing@science.doe.gov,kim.laing@science.doe.gov,"Kimberlie J. Laing, Grant Analyst, 301-903-302...",https://www.fedconnect.net,FedConnect,NaN,NaN
7,131095,Management of HIV-Related Lung Disease and Car...,RFA-HL-12-034,D,G,HL,NaN,93.838,"00,01,02,04,05,06,07,08,11,12,13,20,22,23,25",Other Eligible Applicants include the followin...,...,Synopsis 1,No,4152012.0,FBOWebmaster@OD.NIH.GOV,If you have any problems linking to this fundi...,NIH OER Webmaster&lt;br/&gt;FBOWebmaster@OD.NI...,http://grants.nih.gov/grants/guide/rfa-files/R...,http://grants.nih.gov/grants/guide/rfa-files/R...,NaN,NaN
8,131093,Shared Instrumentation Grant Program (S10),PAR-12-017,D,G,"ED,HL",NaN,"93.351,93.389","06,12,13,20,25",Other Eligible Applicants include the followin...,...,Synopsis 2,No,4212012.0,FBOWebmaster@OD.NIH.GOV,If you have any problems linking to this fundi...,NIH OER Webmaster&lt;br/&gt;FBOWebmaster@OD.NI...,http://grants.nih.gov/grants/guide/pa-files/PA...,http://grants.nih.gov/grants/guide/pa-files/PA...,NaN,NaN
9,196633,Canalway and Riverwalk Projects within the Cit...,NPS-12-NERO-0130,M,CA,"CD,ENV,NR",NaN,15.946,02,NaN,...,Synopsis 1,No,9042012.0,keith_zotti@nps.gov,Lowell Canalway/Riverwalk,Keith Zotti &lt;br/&gt;Partnership Program Spe...,NaN,NaN,NaN,NaN


## Pivot scraping

In [6]:
URL_BASE = 'http://pivot.cos.com/funding/results?'
ENTRIES_PER_PAGE = 25

credentials = {
    'uid':'jlchin@berkeley.edu',
    'pw':'a'
}

N_PAGES_TO_SCRAPE=3

SCRAPED_DATA_FILE_LOCATION = 'scraped_data'

all_rows = []
headers = ['Grant_Name', 'Sponsor_Title', 'Info_url', 'Deadline_Date', 'Certainty',
          'App_Type', 'Mandatory', 'Invite_Only', 'Note']

def get_rows_from_soup(soup):
    rows = []
    t_body = soup.find_all('table', {'id':'funding_results_table'})[0].find_all('tbody')[0]
    for tr in t_body.find_all('tr', recursive=False):
        trs = tr.find_all('td', recursive=False)
        info = trs[1].find_all('dd')[0]
        deadline_info_tds = trs[2].find_all('td')
        money_tr = trs[3]

        grant_name = info.find_all('a')[0].text
        sponsor_title = info.find_all('span')[0].text
        additional_info_url = info.find_all('a')[0]['href']

        #can actually be more than 6

        deadline_date = deadline_info_tds[0].text
        certainty, app_type, mandatory, invite_only, note = [td.text for td in deadline_info_tds[1:6]]
        rows.append([grant_name, sponsor_title, additional_info_url, deadline_date, 
                     certainty, app_type, mandatory, invite_only, note])
    return rows

In [4]:
#http://stackoverflow.com/questions/11892729/how-to-log-in-to-a-website-using-pythons-requests-module
with requests.Session() as s:
    p = s.post(url='https://pivot.cos.com/session/login', data=credentials)

    for i in range(N_PAGES_TO_SCRAPE):
        url = URL_BASE + 'start={}&sort=score%2Bdesc&page={}&searchlist0=full&boolFull=AND&searchfield0a=computer+science&commit=Search+Pivot&full=%28computer+science%29&fq_fundingType%5B%5D=13'.format(i*ENTRIES_PER_PAGE, i+1)

        r = s.get(url=url)
        
        soup = BeautifulSoup(r.content)
        
        all_rows.extend(get_rows_from_soup(soup))

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:318: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#snimissingwarning.
  SNIMissingWarning
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [5]:
df = pd.DataFrame(all_rows, columns=headers)

In [87]:
df.to_csv(scraped_file_loc('test'), sep='~', index=False, encoding='utf-8')

In [88]:
pd.read_csv(scraped_file_loc('test'), sep='~')

,Grant_Name,Sponsor_Title,Info_url,Deadline_Date,Certainty,App_Type,Mandatory,Invite_Only,Note
0,Visiting Researcher Support Program,International Information Science Foundation (...,/funding_opps/47815,"May 31, 2017",Anticipated,Application,Yes,No,NaN
1,Algorithms in the Field (AitF),National Science Foundation (NSF) Directorate ...,/funding_opps/156554,"Jan 26, 2018",Anticipated,Full Proposal,Yes,No,"Full Proposal Window: January 12, 2017 - Janu..."
2,Research Experiences for Teachers (RET) in Eng...,Limited,/funding_opps/77659,"Nov 01, 2017",Anticipated,Full Proposal,Yes,No,NaN
3,Memberships,Institute for Advanced Study (IAS),/funding_opps/119661,"Dec 01, 2017",Anticipated,Application,Yes,No,NaN
4,Emerging Scholar Award,Facebook,/funding_opps/170917,"Feb 28, 2017",Confirmed,Application,Yes,No,"All application materials, including two lette..."
5,Measurement Science and Engineering (MSE) Rese...,United States Department of Commerce (DOC) Nat...,/funding_opps/97522,Continuous,Confirmed,Proposals,Yes,No,Applications will be considered on a continuin...
6,IEEE History Center Summer Internship,Institute of Electrical and Electronics Engine...,/funding_opps/121671,"Mar 01, 2017",Confirmed,Application,Yes,No,NaN
7,Wireless Innovation between Finland and the US...,National Science Foundation (NSF) Directorate ...,/funding_opps/151802,"Oct 17, 2017",Anticipated,Full Proposal,Yes,No,due by 5 p.m. proposer's local time
8,Quantum Information Science,National Science Foundation (NSF) Directorate ...,/funding_opps/159251,"Dec 01, 2017",Anticipated,Full Proposal,Yes,No,NaN
9,United States-Israel Collaboration in Computer...,National Science Foundation (NSF) Directorate ...,/funding_opps/139700,"Feb 15, 2017",Confirmed,Full Proposal,Yes,No,NaN


In [6]:
df

,Grant_Name,Sponsor_Title,Info_url,Deadline_Date,Certainty,App_Type,Mandatory,Invite_Only,Note
0,Visiting Researcher Support Program,International Information Science Foundation (...,/funding_opps/47815,"May 31, 2017",Anticipated,Application,Yes,No,
1,Algorithms in the Field (AitF),National Science Foundation (NSF) Directorate ...,/funding_opps/156554,"Jan 26, 2018",Anticipated,Full Proposal,Yes,No,"Full Proposal Window: January 12, 2017 - Janu..."
2,Research Experiences for Teachers (RET) in Eng...,Limited,/funding_opps/77659,"Nov 01, 2017",Anticipated,Full Proposal,Yes,No,
3,Memberships,Institute for Advanced Study (IAS),/funding_opps/119661,"Dec 01, 2017",Anticipated,Application,Yes,No,
4,Emerging Scholar Award,Facebook,/funding_opps/170917,"Feb 28, 2017",Confirmed,Application,Yes,No,"All application materials, including two lette..."
5,Measurement Science and Engineering (MSE) Rese...,United States Department of Commerce (DOC) Nat...,/funding_opps/97522,Continuous,Confirmed,Proposals,Yes,No,Applications will be considered on a continuin...
6,IEEE History Center Summer Internship,Institute of Electrical and Electronics Engine...,/funding_opps/121671,"Mar 01, 2017",Confirmed,Application,Yes,No,
7,Wireless Innovation between Finland and the US...,National Science Foundation (NSF) Directorate ...,/funding_opps/151802,"Oct 17, 2017",Anticipated,Full Proposal,Yes,No,due by 5 p.m. proposer's local time
8,Quantum Information Science,National Science Foundation (NSF) Directorate ...,/funding_opps/159251,"Dec 01, 2017",Anticipated,Full Proposal,Yes,No,
9,United States-Israel Collaboration in Computer...,National Science Foundation (NSF) Directorate ...,/funding_opps/139700,"Feb 15, 2017",Confirmed,Full Proposal,Yes,No,
